In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

helper_path = os.path.abspath(os.path.join('../helper'))
if helper_path not in sys.path:
    sys.path.append(helper_path)
# !{sys.executable} -m pip install -r ../../requirements.txt

In [4]:
from helper.dataclass import HDDDataset
from helper.algorithms.Stacking_sklearn import StackingSK
from helper.preprocessing import *

## Data preprocessing

In [5]:
hdd_dataset = HDDDataset.read_csv('ST14000NM001G.csv')

In [6]:
preprocessor = Preprocessing(hdd_dataset)
preprocessor.clear_unused_data()
preprocessor.add_target_column()
preprocessor.prepare_train_df()
preprocessor.add_time_features()
preprocessor.train_test_val_split(train_size=0.9, test_size=0.1, val_size=0) #sampling_strat=0.05, oversampling='Default')
preprocessor.normalize_data(method='YJ', inplace=True)
preprocessor.drop_unimportant_features(drop=15)
preprocessor.rescale_types()

Clearing unused columns...
Adding target column...
Prepairing train dataset...
Adding time features to unsplitted dataset...
Splitting train dataset...
Normalizing splitted dataset with YJ...
    Удаляем не нужные признаки...     
failure 
diff_smart_5_raw_1 
diff_smart_199_raw_1 
smart_197_raw_normalized 
diff_smart_9_raw_1 
diff_smart_240_raw_1 
diff_smart_188_raw_1 
smart_199_raw_normalized 
diff_smart_242_raw_1 
diff_smart_241_raw_1 
smart_9_raw_normalized 
diff_smart_192_raw_1 
smart_242_raw_normalized 
smart_188_raw_normalized 
smart_241_raw_normalized 


In [7]:
import gc
gc.collect()

0

## Training

In [8]:
model = StackingSK()
model.fit(preprocessor.X_train, preprocessor.y_train)
preds = model.predict(preprocessor.X_test)

## Testing

In [9]:
from helper.metrics import *
from helper.saver import Saver
metrics = Metrics.get_metrics(preds, preprocessor.y_test)
print(metrics)
# Посмотрим метрики

{'accuracy': 0.999748639030621, 'precision': 0.9796511627906976, 'recall': 0.6556420233463035, 'f1': 0.7855477855477856, 'auc_roc': 0.8278162269906975}


In [10]:
from sklearn.metrics import classification_report
print(classification_report(preprocessor.y_test, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    731501
           1       0.98      0.66      0.79       514

    accuracy                           1.00    732015
   macro avg       0.99      0.83      0.89    732015
weighted avg       1.00      1.00      1.00    732015



## Logging

In [11]:
Saver.save(model, preprocessor, metrics, save_csv=False)